Calibrate: MOT shutter delay, delay of OP shutters

In [2]:
exposure=0.002

timeline=tl.stack(
    ex.init(t=-2,
            shutter_imaging=0,AOM_imaging=1,trigger_camera=0
            ),
    ex.MOT(),
    ex.MOT_detunedGrowth(),
    ex.molasses(),
    ex.OP(),
    ex.magneticTrapping(),
    ex.finish(wait=2,MOT_ON=True,
              shutter_imaging=0,AOM_imaging=1,trigger_camera=0
              ),
#    trigger_camera(15.5,exposure,"AI"),
    )

timeline=di.imaging_absorption(10e-3,
    #15.5,
    exposure,exposureBlow=1e-2,delayBg=50e-3,delayLi=1,
    anchorToContext="magneticTrapping",
    timeline=timeline
    )

In [3]:
dp.display_new(timeline,variables = ["Anchor","shutter_MOT","AOM_MOT","shutter_repump","AOM_repump",
                                     "shutter_OP001","shutter_OP002","AOM_OP","coil_MOTlower__A","coil_MOTupper__A","lockbox_MOT__MHz",
                                     "shutter_imaging","AOM_imaging","trigger_camera"
                                     ])

(<Figure size 750x750 with 4 Axes>,
 array([<Axes: ylabel='Current [A]'>, <Axes: ylabel='Frequency [MHz]'>,
        <Axes: ylabel='Digital channels'>], dtype=object))

In [5]:
#ad.output(
ad.add(timeline,connections,ex.devices).iloc[-1]["cycle"]#)

np.int64(2147483648)

# Imports

In [1]:
import sys
sys.path.append("..")

import pandas as pd

from wigner_time import timeline as tl
from wigner_time import adwin as adwin
from wigner_time import connection as con
from wigner_time import display as dp
from wigner_time import adwin as ad
import wigner_time

import experiment as ex
import diagnostics as di

%matplotlib qt

# Tests old

## Testing the difference between `globalRelative=True/False` in `next`

In [ ]:
timeline=tl.stack(
    init(t=-2,coil_MOTlower__A=.0,coil_MOTupper__A=.0),
    tl.next(coil_MOTlower__A=-3.0, t=1., time_start=0., context="any"),
    tl.next(coil_MOTupper__A=-4.0, t=1.5, time_start=0.5),
    tl.next(lockbox_MOT__MHz=-5, t=1.5, globalRelative=False)
)

In [ ]:
tl.display(timeline)#,variables=["coil_MOTlower__A","coil_MOTupper","lockbox_MOT__MHz"])

In [ ]:
timeline=tl.stack(
    init(t=-2#,shutter_imaging=0,AOM_imaging=1,trigger_camera=0
        ),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    tl.wait(1),
    tl.next(
        lockbox_MOT__MHz=0,
        coil_MOTlower__A=-1.0,
        coil_MOTupper__A=-0.98,
        t=.1,
        fargs={"ti": 3},context='sanitize'
    ),
    tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
)

In [ ]:
prev=tl.previous(timeline,"coil_MOTlower__A")
prev.values[0]=1#["value"]
prev

# Thomas’s solution

In [ ]:
defaults = Munch()

# NOTE: MOTplus coils are part of the compensation and so should default to the compensation values.
defaults.MOT = Munch(
    lockbox_MOT__V=0.0,
    shutter_MOT=1, # TODO: why the shutter values here?
    shutter_repump=1,
    coil_MOTlower__A=-1.0,
    coil_MOTupper__A=-0.98,
)

defaults.molasses = Munch(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    shift__MHz=-80,
    fraction_ramp_duration=0.9,
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
)

defaults.magnetic = Munch(
    delay=1e-3,
    quadrupole=Munch(duration_ramp=50e-6, coil_MOTlower__A=-1.8, coil_MOTupper__A=-1.7),
    strong=Munch(duration_ramp=3e-3, coil_MOTlower__A=-4.8, coil_MOTupper__A=-4.7),
)

defaults.finish = Munch(set={}, ramp={})
defaults.finish.set = Munch(
    AOM_MOT=1,
    AOM_repump=1,
    AOM_repump__V=5,
    AOM_imaging=1,
    AOM_OP_aux=0,
    AOM_OP=1,
    AOM_science=1,
    AOM_science__V=0.0,
    AOM_ref=1,
    shutter_MOT=1,
    shutter_repump=1,
    shutter_imaging=0,
    shutter_OP001=0,
    shutter_OP002=1,
    shutter_science=1,
    shutter_transversePump=0,
    shutter_coupling=0,
    trigger_TC__V=0,
    photodiode__V=0,
)
defaults.finish.ramp = Munch(
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
    coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
    coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
    coil_compensationX__A=constants.Bfield_compensation_X__A,
    coil_compensationY__A=constants.Bfield_compensation_Y__A,
    lockbox_MOT__V=lock_box.to_V(0.0),
)

In [ ]:
def init():
    """
    Creates an experimental timeline for the initialization of every relevant variable.
    """
    return tl.create(
        lockbox_MOT__V=0.0,
        coil_compensationX__A=constants.Bfield_compensation_X__A,
        coil_compensationY__A=constants.Bfield_compensation_Y__A,
        coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
        coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
        AOM_MOT=1,
        AOM_repump=1,
        AOM_OP_aux=0,
        AOM_OP=1,
        shutter_MOT=0,
        shutter_repump=0,
        shutter_OP001=0,
        shutter_OP002=1,
        context="ADwin_LowInit",
    )

def MOT(
    detuning__MHz=-5,
    duration=10.0,
    durationFinal=1.0,
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.MOT,
    timeline=init(),
    context="MOT",
):
    """
    Creates a Magneto-Optical Trap.

    """
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    return tl.update(
        tl.create(
            shutter_MOT=_variables.shutter_MOT,
            shutter_repump=_variables.shutter_repump,
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            # ===
            timeline=timeline,
            context=context,
        ),
        tl.next(
            lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=durationFinal, time_start=duration-durationFinal, context="MOT_grow"
        ),
    )


def molasses(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    detuning__MHz=-80,
    # ===
    # Specific values above ↑
    # Repeated values below ↓
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.molasses,
    timeline=MOT(),
    context="molasses",
):
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    if duration_ramp >= duration_cooling:
        raise ValueError("duration_ramp should be smaller than duration_cooling!")

    return tl.update(
        tl.create(
            AOM_MOT=[_time_start + duration_cooling, 0],
            shutter_MOT=[_time_start + duration_cooling - constants.lag_MOTshutter, 0],
            timeline=timeline,
            context=context,
        ),
        tl.next(
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            t=duration_ramp,
        ),
        tl.next(lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=duration_ramp),
    )

In [ ]:
init()

In [ ]:
tl.display(molasses(timeline=MOT(variables={"coil_MOTlower__A":-.5})))

# Params layer

In [ ]:
params_default = Munch(
    MOT=Munch(
        duration_grow=15.0,
        duration_final=100e-3, # should be smaller than duration_grow
        shift__MHz=-5.0,
        duration_ramp=10e-3,
        coil_lower__A=-1,
        coil_upper__A=-0.98, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    molasses=Munch(
        duration_cooling=5e-3,
        duration_ramp=1e-3, # should be smaller than duration_cooling
        shift__MHz=-80,
        fraction_ramp_duration=0.9,
        ),
    OP=Munch(
        duration_exposure=80e-6,
        duration_ramp_coils_dipole=500e-6,
        homogeneous_field__A=-0.12,
    ),
    trapping_magnetic=Munch(
        delay_magnetic_trap=1e-3,
        duration_ramp_coils_quadrupole=50e-6,
        duration_ramp_coils_strengthen=3e-3,
        lowerMOT_quadrupole__A=-1.8,
        upperMOT_quadrupole__A=-1.7, # TODO why the asymmetry? this was probably part of magnetic compensation
        lowerMOT_strengthen__A=-4.8,
        upperMOT_strengthen__A=-4.7, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    transport_magnetic=Munch(
        delay=5e-3,
        duration=150e-3,
        lowerMOT__A=-0.3121632,
        upperMOT__A=-4.748368,
        lowerMOTPlus__A=-4.648368,
        upperMOTPlus__A=-4.748368,
        parameter_tanh=2.25,
    ),
    finish=Munch(
        duration_until_finish = 0,
        duration_ramp = 1e-3,
        lockbox_initial__V = 0,
    )
)

In [ ]:
def init():
    return preparation_functions.init()

def MOT(tline, time, params,context=""):
    tline = preparation_functions.MOT(tline, time, params.MOT.lowerMOT__A, params.MOT.upperMOT__A)
    tline = preparation_functions.MOT_detunedGrowth(tline, time+params.MOT.duration_grow-params.MOT.duration_final, params.MOT.duration_ramp, 
                                             params.MOT.shift__MHz,pt=3)
    return tline, time+params.MOT.duration_grow

def molasses(tline, time, params):
    tline = preparation_functions.molasses(tline, time, params.molasses.duration_cooling, params.molasses.duration_ramp, params.molasses.duration_ramp, 
                                            params.molasses.shift__MHz)
    return tline, time+params.molasses.duration_cooling

def OP(tline, time, params):
    #def OP(tline, time, duration_exposure, duration_ramp_coils_dipole, duration_shutter_on, homogenous_field__A,coil_parameter_tanh=3):
    tline = preparation_functions.OP(tline, time, params.OP.duration_exposure, params.OP.duration_ramp_coils_dipole, params.OP.homogenous_field__A)
    return tline, time+params.OP.duration_exposure

def magnetic_field(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_quadrupole, params.trapping_magnetic.lowerMOT_quadrupole__A,
                                                    params.trapping_magnetic.upperMOT_quadrupole__A,context="trapping_magnetic")
    time = time+params.trapping_magnetic.duration_ramp_coils_quadrupole
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_strengthen, params.trapping_magnetic.lowerMOT_strengthen__A,
                                             params.trapping_magnetic.upperMOT_strengthen__A,context="trapping_magnetic")
    return tline, time+params.trapping_magnetic.duration_ramp_coils_strengthen

def transport_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.transport_magnetic.duration, params.transport_magnetic.lowerMOT__A,
                                            params.transport_magnetic.upperMOT__A,params.transport_magnetic.lowerMOTPlus__A,params.transport_magnetic.upperMOTPlus__A,
                                            context = "magnetic_transport")
    return tline, time+params.transport_magnetic.duration

def switch_off_trap_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.switch_off_trap_magnetic.duration, 0,0)
    return tline, time+params.switch_off_trap_magnetic.duration

#parameter??
def transport_back_magnetic(tline, time, params):
    tline = preparation_functions.transport_back_magnetic(tline, time, params.transport_back_magnetic.duration, params.transport_back_magnetic.lowerMOT_strengthen__A, params.transport_back_magnetic.upperMOT_strengthen__A)
    return tline, time+params.transport_back_magnetic.duration

def finish(tline, time, params):
    return preparation_functions.finish(tline, time, params.finish.duration_ramp, params.finish.lockbox_initial__V, params.finish.science_AOM__V, params.finish.coil_lowerMOT__A, params.finish.coil_upperMOT__A)

In [ ]:
def prepare_atoms(params):
    tline = init()
    tline,timeMOT = MOT(tline, 0, params)
    tline,timemolasses = molasses( *MOT(tline, 0, params), params)
    tline,timeOP = OP(tline, timemolasses, params)
    tline,timeTrapping = magnetic_field(tline, timeOP+params.trapping_magnetic.delay_magnetic_trap, params)
    tline = finish(tline, timeTrapping, params)
    return tline

In [ ]:
params=deepcopy(params_default)
params.finish.duration_until_finish = 500e-3

tline = prepare_atoms(params)

In [ ]:
%matplotlib qt
tl.display(tline,variables=['lockbox_MOT__V','AOM_MOT','AOM_repump','shutter_repump','AOM_repump__V'],xlim=[15,15.05])